<h3>Check the speed of cars in a video</h3>
<p>This notebook is used to practice recognizing, tracking, and estimating the speed using videos of cars and trucks passing by my window, without having to wait for a car to drive by.
<h4>Pseudocode:</h4>
<p>Convert feet per second to mph: 1 ft/s = 0.6818181818 mi/h

In [25]:
import os
import cv2
import time
from datetime import datetime
path_to_videos = 'cars-and-trucks/'
car_cascade = cv2.CascadeClassifier('cars.xml')

In [26]:
def get_next_filename(file_num):
    path_to_videos = 'cars-and-trucks/'
    files = os.listdir(path_to_videos)
    this_file_num = 0
    for file in files:
        if os.path.isfile(os.path.join(path_to_videos, file)):
            this_file_num = this_file_num + 1
            file_name = os.path.join(path_to_videos, file)
            if file_num == this_file_num:
                return (file_name)

In [27]:
def speed_in_mph(x1,x2,f1,f2):
    if x1 == 0:
        return(None)
    speed_of_car = 0
    pixels_traveled = abs(x1 - x2) # first to last notice of a car
#     print('in speed_in_mph')
#     print('pixels traveled: ',pixels_traveled)
    frames_with_cars = abs(f1 - f2) # from containing the first and last notice of a car
#     print('frames with cars: ', frames_with_cars)
    if frames_with_cars > 0:
        time_elapsed = frames_with_cars / 30 # number of seconds from first frame with car to last frame with car
#         print ('Time elapsed: ',time_elapsed,' seconds')
        field_traveled = pixels_traveled / 320  # what portion of the field of view was the car noticed
#         print ('portion of the view traveled: ',field_traveled*100,'%')
        distance_traveled = field_traveled * 65 # feet of travel - averaged of near and far lanes 50 & 45' respectively
#         print ('Distance traveled: ',distance_traveled,' in feet')
        feet_per_second = distance_traveled / time_elapsed
#         print('Feet per second: ',feet_per_second)
        speed_of_car = feet_per_second * 0.6818181818 # convert feet per second the miles per hour
#         print('Speed of car: ',speed_of_car)
    return (speed_of_car)

In [28]:
def found_a_car(location):
    big_x = 0
    little_x = 640
    frame_big_x = 0
    frame_little_x = 0
    speed = 0
    p = 0
    for (x,y,w,h,frames) in location:
            p = x + w / 2 # a point midway through the rectangle 
            if p>big_x:
                big_x = p
                frame_big_x = frames
            if p<little_x:
                little_x = p
                frame_little_x = frames
    if big_x != 0:
        speed = round(speed_in_mph(big_x,little_x,frame_big_x,frame_little_x),3) # speed not set
    return(speed)

In [29]:
font = cv2.FONT_HERSHEY_SIMPLEX
def process_video(file_name):
    cap = cv2.VideoCapture(file_name)
    if cap.isOpened() == False:
        print('Error - wrong file name or codec used')
    frames = 0
    big_x = 0
    little_x = 640
    frame_big_x = 0
    how_fast = 0
    frame_little_x = 0
    location = []
    while cap.isOpened():
        ret,frame = cap.read()
        if ret == True:
            # here is where you need to process each frame
            # detect the vehicle in a single frame
            frames += 1 # increment the number of frames so far, each one represents 1/30th of a second
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detects cars of different sizes in the input image
            cars = car_cascade.detectMultiScale(gray, 1.1, 2)
            for (x,y,w,h) in cars:
                location.append([x,y,w,h,frames])
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            time.sleep(.005)
            cv2.putText(frame,text=str(frames),org=(10,450),fontFace=font,fontScale=1,color=(0,255,0),thickness=2,lineType=cv2.LINE_AA)
            cv2.imshow('frame',frame)
            if cv2.waitKey(10) & 0xFF == 27:
                break
        else:
            break
        how_fast = found_a_car(location)
    print('estimated speed was ',how_fast)
    cap.release()

In [36]:
this_file_num = 13
this_file = get_next_filename(this_file_num)
this_file = 'cars-and-trucks/08-17-2021 11:04:24.mp4'
# while(this_file != None):
print('Showing video file named ',this_file)
process_video(this_file)
this_file_num += 1
this_file = get_next_filename(this_file_num)
cv2.destroyAllWindows()

Showing video file named  cars-and-trucks/08-17-2021 11:04:24.mp4
estimated speed was  28.118


<h3>Check the recorded video for accuracy</h3>
<p>Play the video back at 2 frames a second and count the frames from the start to the finish. Watch the back of the car and determine how many frames it appears in. Forget the front of the car.</p>
If it appears in 30 frames and covers 40 feet, that's 20 feet per second.


In [31]:
# width_of_screen = 8 # inches across
# width_of_car_travelled = 6 # inches from start of video
# pixels_traveled = 640 / (8/6) # first to last notice of a car
# print('pixels car traveled: ',pixels_traveled)
# frames_with_cars = 26 # from containing the first and last notice of a car
# time_elapsed = frames_with_cars / 30 # number of seconds from first frame with car to last frame with car
# print ('Time elapsed: ',time_elapsed,' seconds')
# field_traveled = pixels_traveled / 640  # what portion of the field of view was the car noticed
# print ('portSpeed of car:  27.39448051875ion of the view traveled: ',field_traveled*100,'%')
# distance_traveled = field_traveled * 50 # feet of travel - averaged of near and far lanes 50 & 45' respectively
# print ('Distance traveled: ',distance_traveled,' in feet')
# feet_per_second = distance_traveled / time_elapsed
# print('Feet per second: ',feet_per_second)
# speed_of_car = feet_per_second * 0.6818181818 # convert feet per second the miles per hour
# print('Speed of car: ',speed_of_car)